<a href="https://colab.research.google.com/github/victor-roris/ML-learning/blob/master/Ranking/DirectRanker_(Fails_TF_incompatibilites).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DirectRanker

 - [GitHub](https://github.com/kramerlab/direct-ranker)
 - [Paper](https://arxiv.org/pdf/1909.02768v1.pdf)

Pairwise learning to rank approach based on a neural net, called DirectRanker, that generalizes the RankNet architecture.

**NOTE**: I wasn't able to run the code. I think it can be a problem with the dataset I used and tensorflow incompatibilities. Colab use the tf v2 but if I change to v1 it fails too. I suppose they use a very specific version but they don't provide it.  

## Installation

The project provided is only code, no library. This code `import`s internal packages using the project name: `supplementary_code_direct_ranker`. 

My strategy is the following:

In [1]:
!git clone https://github.com/kramerlab/direct-ranker.git
!mv direct-ranker supplementary_code_direct_ranker

Cloning into 'direct-ranker'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.


In [1]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/contents/supplementary_code_direct_ranker')

## Imports

In [2]:
from __future__ import print_function

import sys
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split

Imports fail in the DirectRanker code because tensorflow api. The importantion can work if you use the tensorflow version 1. But then it fails again because new features. I suppose they are using a very specific version, but they not provide it. 

To change tensorflow version, modify the import in the corresponding files:
```python
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
```

In [ ]:
import tensorflow as tf

from supplementary_code_direct_ranker.DirectRanker import directRanker
from supplementary_code_direct_ranker.helpers import readData, nDCGScorer_cls, MAP_cls

## Load dataset

I use the `msrank` dataset. The proposed dataset is difficult to manage.

In [ ]:
!pip install catboost

In [6]:
from catboost.datasets import msrank_10k
train_df, test_df = msrank_10k()

X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values
queries_train = train_df[1].values

X_test = test_df.drop([0, 1], axis=1).values
y_test = test_df[0].values
queries_test = test_df[1].values

## Define model

In [7]:
def lambda_cost(nn, y0):
    return tf.reduce_mean(tf.log(1+tf.exp(nn))-nn)


# Load directRanker, train, and test
dr = directRanker(
    feature_activation=tf.nn.tanh,
    ranking_activation=tf.nn.tanh,
    # max_steps=10000,
    # For debugging
    #cost=lambda_cost,
    max_steps=10000,
    print_step=500,
    start_batch_size=3,
    end_batch_size=5,
    start_qids=20,
    end_qids=100,
    feature_bias=True,
    hidden_layers=[100, 50, 5]
)

## Train model

If we change the tf version, then we have problems with the data. Can be a problem with the dataset I use?

In [ ]:
dr.fit(X_train, y_train, ranking=True)

## Evaluation

In [ ]:
nDCGScorer_cls(dr, X_test, y_test, at=10)
MAP_cls(dr, x_test, y_test)